In [ ]:
import pandas as pd
import sys

sys.path.append("./pysubgroup/src/")
import pysubgroup as ps

In [5]:
path = "/home/gabriel/.cache/kagglehub/datasets/arjunbhasin2013/ccdata/versions/1/CC GENERAL.csv"
creditData = pd.read_csv(path)

In [6]:
# Pré-processamento
creditData = creditData.dropna().round(2)

In [ ]:
# SD 1: alvo numérico - colunas comportamentais e financeiras - regras grandes (8) + punição 1.0

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=8,
    qf=ps.StandardQFNumeric(a=1.0, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,51510.573414,PURCHASES>=1451.50,1728,8636,44.842593,15.033233,40.563630,25.17901,34.0,7.0,358,358,1,0,2.982897,4.857143
1,50253.594604,PURCHASES>=1451.50 AND TENURE>=12,1607,8636,46.304916,15.033233,41.257147,25.17901,35.0,7.0,358,358,1,0,3.080170,5.000000
2,49070.945113,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50,1446,8636,48.968880,15.033233,42.435123,25.17901,37.0,7.0,358,358,1,0,3.257375,5.285714
3,48683.346688,PURCHASES_FREQUENCY>=1.0,2126,8636,37.932267,15.033233,37.205515,25.17901,26.0,7.0,358,358,0,0,2.523228,3.714286
4,48633.627721,PURCHASES_FREQUENCY>=1.0 AND TENURE>=12,1937,8636,40.140940,15.033233,37.881650,25.17901,28.0,7.0,358,358,0,0,2.670147,4.000000
5,48347.573529,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50 ...,1397,8636,49.641374,15.033233,42.684886,25.17901,38.0,7.0,358,358,1,0,3.302109,5.428571
6,47364.796897,BALANCE_FREQUENCY>=1.0 AND PURCHASES_FREQUENCY...,1962,8636,39.174312,15.033233,38.172408,25.17901,26.0,7.0,358,358,0,0,2.605847,3.714286
7,47274.512969,BALANCE_FREQUENCY>=1.0 AND PURCHASES_FREQUENCY...,1790,8636,41.443575,15.033233,38.826773,25.17901,29.0,7.0,358,358,0,0,2.756797,4.142857
8,46220.573414,INSTALLMENTS_PURCHASES>=620.84,1728,8636,41.781250,15.033233,40.122421,25.17901,30.0,7.0,358,358,0,0,2.779259,4.285714
9,45345.760769,INSTALLMENTS_PURCHASES>=620.84 AND TENURE>=12,1602,8636,43.338951,15.033233,40.831502,25.17901,31.0,7.0,358,358,0,0,2.882876,4.428571


In [ ]:
# SD 2: alvo numérico - colunas comportamentais e financeiras - regras grandes (8) + punição 0.5

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=8,
    qf=ps.StandardQFNumeric(a=0.5, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,1376.919934,PURCHASES>=1451.50 AND PURCHASES_FREQUENCY>=1.0,981,8636,58.994903,15.033233,44.375460,25.17901,48.0,7.0,358,358,6,0,3.924299,6.857143
1,1374.712320,PURCHASES>=1451.50 AND PURCHASES_FREQUENCY>=1....,953,8636,59.564533,15.033233,44.559359,25.17901,48.0,7.0,358,358,12,0,3.962191,6.857143
2,1372.529295,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50 ...,942,8636,59.752654,15.033233,44.916074,25.17901,49.0,7.0,358,358,6,0,3.974704,7.000000
3,1370.005766,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50 ...,917,8636,60.274809,15.033233,45.070849,25.17901,49.0,7.0,358,358,12,0,4.009438,7.000000
4,1339.369029,INSTALLMENTS_PURCHASES>=620.84 AND PURCHASES>=...,767,8636,63.395046,15.033233,47.380500,25.17901,51.0,7.0,358,358,6,0,4.216993,7.285714
5,1336.844301,INSTALLMENTS_PURCHASES>=620.84 AND PURCHASES>=...,748,8636,63.913102,15.033233,47.517874,25.17901,51.0,7.0,358,358,12,0,4.251454,7.285714
6,1334.651225,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,735,8636,64.262585,15.033233,47.992946,25.17901,52.0,7.0,358,358,6,0,4.274702,7.428571
7,1331.641489,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,717,8636,64.764296,15.033233,48.135866,25.17901,52.0,7.0,358,358,12,0,4.308075,7.428571
8,1329.600072,ONEOFF_PURCHASES_FREQUENCY>=0.42 AND PURCHASES...,691,8636,65.613603,15.033233,44.704883,25.17901,53.0,7.0,358,358,12,0,4.364570,7.571429
9,1324.520460,BALANCE_FREQUENCY>=1.0 AND ONEOFF_PURCHASES_FR...,669,8636,66.242152,15.033233,45.121043,25.17901,54.0,7.0,358,358,12,0,4.406381,7.714286


In [11]:
# SD 3: alvo numérico - colunas comportamentais e financeiras - regras grandes (8) + punição 0.3

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=8,
    qf=ps.StandardQFNumeric(a=0.3, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,389.750632,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,240,8636,90.320833,15.033233,53.076450,25.17901,79.5,7.0,358,358,22,0,6.008078,11.357143
1,389.750632,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,240,8636,90.320833,15.033233,53.076450,25.17901,79.5,7.0,358,358,22,0,6.008078,11.357143
2,389.132812,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,299,8636,85.404682,15.033233,53.769732,25.17901,71.0,7.0,358,358,15,0,5.681059,10.142857
3,389.132812,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,299,8636,85.404682,15.033233,53.769732,25.17901,71.0,7.0,358,358,15,0,5.681059,10.142857
4,388.384185,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,231,8636,90.922078,15.033233,52.878960,25.17901,80.0,7.0,358,358,22,0,6.048072,11.428571
5,388.384185,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,231,8636,90.922078,15.033233,52.878960,25.17901,80.0,7.0,358,358,22,0,6.048072,11.428571
6,388.346315,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,329,8636,83.276596,15.033233,49.667460,25.17901,73.0,7.0,358,358,19,0,5.539500,10.428571
7,388.346315,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,329,8636,83.276596,15.033233,49.667460,25.17901,73.0,7.0,358,358,19,0,5.539500,10.428571
8,388.237078,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,248,8636,89.294355,15.033233,52.647853,25.17901,78.5,7.0,358,358,22,0,5.939797,11.214286
9,388.237078,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,248,8636,89.294355,15.033233,52.647853,25.17901,78.5,7.0,358,358,22,0,5.939797,11.214286


In [ ]:
# SD 4: alvo numérico - colunas comportamentais e financeiras - regras grandes (8) + punição 0.0

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=8,
    qf=ps.StandardQFNumeric(a=0.0, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE_TRX: [0:1[ A...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
1,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE_FREQUENCY: [...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
2,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE: [0.0:14.22[...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
3,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE: [0.0:14.22[...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
4,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE: [0.0:14.22[...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
5,328.966767,BALANCE>=2613.12 AND CASH_ADVANCE: [0.0:14.22[...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
6,328.966767,BALANCE>=2613.12 AND BALANCE_FREQUENCY>=1.0 AN...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
7,328.966767,BALANCE>=2613.12 AND BALANCE_FREQUENCY>=1.0 AN...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
8,328.966767,BALANCE>=2613.12 AND BALANCE_FREQUENCY>=1.0 AN...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857
9,328.966767,BALANCE>=2613.12 AND BALANCE_FREQUENCY>=1.0 AN...,1,8636,344.0,15.033233,0.0,25.17901,344.0,7.0,344,358,344,0,22.882636,49.142857


In [12]:
# SD 5: alvo numérico - colunas comportamentais e financeiras - regras pequenas (3) + punição 1.0

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=3,
    qf=ps.StandardQFNumeric(a=1.0, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,51510.573414,PURCHASES>=1451.50,1728,8636,44.842593,15.033233,40.563630,25.17901,34.0,7.0,358,358,1,0,2.982897,4.857143
1,50253.594604,PURCHASES>=1451.50 AND TENURE>=12,1607,8636,46.304916,15.033233,41.257147,25.17901,35.0,7.0,358,358,1,0,3.080170,5.000000
2,49070.945113,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50,1446,8636,48.968880,15.033233,42.435123,25.17901,37.0,7.0,358,358,1,0,3.257375,5.285714
3,48683.346688,PURCHASES_FREQUENCY>=1.0,2126,8636,37.932267,15.033233,37.205515,25.17901,26.0,7.0,358,358,0,0,2.523228,3.714286
4,48633.627721,PURCHASES_FREQUENCY>=1.0 AND TENURE>=12,1937,8636,40.140940,15.033233,37.881650,25.17901,28.0,7.0,358,358,0,0,2.670147,4.000000
5,48347.573529,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50 ...,1397,8636,49.641374,15.033233,42.684886,25.17901,38.0,7.0,358,358,1,0,3.302109,5.428571
6,47364.796897,BALANCE_FREQUENCY>=1.0 AND PURCHASES_FREQUENCY...,1962,8636,39.174312,15.033233,38.172408,25.17901,26.0,7.0,358,358,0,0,2.605847,3.714286
7,47274.512969,BALANCE_FREQUENCY>=1.0 AND PURCHASES_FREQUENCY...,1790,8636,41.443575,15.033233,38.826773,25.17901,29.0,7.0,358,358,0,0,2.756797,4.142857
8,46220.573414,INSTALLMENTS_PURCHASES>=620.84,1728,8636,41.781250,15.033233,40.122421,25.17901,30.0,7.0,358,358,0,0,2.779259,4.285714
9,45345.760769,INSTALLMENTS_PURCHASES>=620.84 AND TENURE>=12,1602,8636,43.338951,15.033233,40.831502,25.17901,31.0,7.0,358,358,0,0,2.882876,4.428571


In [14]:
# SD 6: alvo numérico - colunas comportamentais e financeiras - regras pequenas (3) + punição 0.5

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=3,
    qf=ps.StandardQFNumeric(a=0.5, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,1376.919934,PURCHASES>=1451.50 AND PURCHASES_FREQUENCY>=1.0,981,8636,58.994903,15.033233,44.375460,25.17901,48.0,7.0,358,358,6,0,3.924299,6.857143
1,1374.712320,PURCHASES>=1451.50 AND PURCHASES_FREQUENCY>=1....,953,8636,59.564533,15.033233,44.559359,25.17901,48.0,7.0,358,358,12,0,3.962191,6.857143
2,1372.529295,BALANCE_FREQUENCY>=1.0 AND PURCHASES>=1451.50 ...,942,8636,59.752654,15.033233,44.916074,25.17901,49.0,7.0,358,358,6,0,3.974704,7.000000
3,1339.369029,INSTALLMENTS_PURCHASES>=620.84 AND PURCHASES>=...,767,8636,63.395046,15.033233,47.380500,25.17901,51.0,7.0,358,358,6,0,4.216993,7.285714
4,1329.600072,ONEOFF_PURCHASES_FREQUENCY>=0.42 AND PURCHASES...,691,8636,65.613603,15.033233,44.704883,25.17901,53.0,7.0,358,358,12,0,4.364570,7.571429
5,1316.068741,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,630,8636,67.466667,15.033233,46.377519,25.17901,54.0,7.0,358,358,10,0,4.487835,7.714286
6,1309.845815,BALANCE_FREQUENCY>=1.0 AND INSTALLMENTS_PURCHA...,955,8636,57.418848,15.033233,46.104689,25.17901,45.0,7.0,358,358,2,0,3.819461,6.428571
7,1309.693666,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,512,8636,72.914062,15.033233,47.323195,25.17901,61.0,7.0,358,358,15,0,4.850192,8.714286
8,1306.724842,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,670,8636,65.516418,15.033233,45.809100,25.17901,52.5,7.0,358,358,8,0,4.358106,7.500000
9,1304.297365,ONEOFF_PURCHASES>=834.80 AND PURCHASES>=1451.5...,715,8636,63.811189,15.033233,44.516635,25.17901,52.0,7.0,358,358,9,0,4.244675,7.428571


In [ ]:
# SD 7: alvo numérico - colunas comportamentais e financeiras - regras pequenas (3) + punição 0.3

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=3,
    qf=ps.StandardQFNumeric(a=0.3, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()

[BALANCE<89.58, BALANCE: [89.58:512.01[, BALANCE: [512.01:1247.42[, BALANCE: [1247.42:2613.12[, BALANCE>=2613.12, BALANCE_FREQUENCY<0.82, BALANCE_FREQUENCY: [0.82:1.0[, BALANCE_FREQUENCY>=1.0, PURCHASES<0.0, PURCHASES: [0.0:223.34[, PURCHASES: [223.34:577.98[, PURCHASES: [577.98:1451.5[, PURCHASES>=1451.5, ONEOFF_PURCHASES<0.0, ONEOFF_PURCHASES: [0.0:0.01[, ONEOFF_PURCHASES: [0.01:177.0[, ONEOFF_PURCHASES: [177.0:834.8[, ONEOFF_PURCHASES>=834.8, INSTALLMENTS_PURCHASES<0.0, INSTALLMENTS_PURCHASES: [0.0:1.95[, INSTALLMENTS_PURCHASES: [1.95:224.54[, INSTALLMENTS_PURCHASES: [224.54:620.84[, INSTALLMENTS_PURCHASES>=620.84, CASH_ADVANCE<0.0, CASH_ADVANCE: [0.0:14.22[, CASH_ADVANCE: [14.22:254.45[, CASH_ADVANCE: [254.45:1600.76[, CASH_ADVANCE>=1600.76, PURCHASES_FREQUENCY<0.0, PURCHASES_FREQUENCY: [0.0:0.27[, PURCHASES_FREQUENCY: [0.27:0.67[, PURCHASES_FREQUENCY: [0.67:1.0[, PURCHASES_FREQUENCY>=1.0, ONEOFF_PURCHASES_FREQUENCY<0.0, ONEOFF_PURCHASES_FREQUENCY: [0.0:0.08[, ONEOFF_PURCHASES_FREQ

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,380.355032,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,403,8636,77.925558,15.033233,47.410298,25.17901,67.0,7.0,358,358,19,0,5.183553,9.571429
1,376.110740,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,512,8636,72.914062,15.033233,47.323195,25.17901,61.0,7.0,358,358,15,0,4.850192,8.714286
2,373.736893,ONEOFF_PURCHASES_FREQUENCY>=0.42 AND PURCHASES...,448,8636,74.899554,15.033233,46.643655,25.17901,64.0,7.0,358,358,14,0,4.982265,9.142857
3,372.952910,ONEOFF_PURCHASES_FREQUENCY>=0.42 AND PAYMENTS>...,310,8636,81.751613,15.033233,50.908706,25.17901,71.0,7.0,358,358,19,0,5.438059,10.142857
4,372.829281,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,407,8636,76.498771,15.033233,51.001945,25.17901,63.0,7.0,358,358,13,0,5.088644,9.000000
5,372.652542,ONEOFF_PURCHASES>=834.80 AND ONEOFF_PURCHASES_...,405,8636,76.560494,15.033233,48.013932,25.17901,66.0,7.0,358,358,14,0,5.092750,9.428571
6,371.671530,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,501,8636,72.604790,15.033233,47.819808,25.17901,60.0,7.0,358,358,13,0,4.829619,8.571429
7,369.380732,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,411,8636,75.751825,15.033233,47.970101,25.17901,65.0,7.0,358,358,9,0,5.038958,9.285714
8,367.147236,INSTALLMENTS_PURCHASES>=620.84 AND ONEOFF_PURC...,558,8636,70.094982,15.033233,47.775760,25.17901,57.0,7.0,358,358,11,0,4.662669,8.142857
9,366.579952,PAYMENTS>=2373.90 AND PURCHASES>=1451.50 AND P...,557,8636,70.039497,15.033233,50.441340,25.17901,58.0,7.0,358,358,11,0,4.658978,8.285714


**Sobre os valores de a**

Ao caminhar pelos valores de a na direção de 1.0 -> 0.0, subgrupos de tamanho menor e com diferenças mais extremas na média em relação à população (para compensar a menor recompensação pelo seu pequeno tamanho) são retornados.

Aparentemente, subgrupos menores são descritos por um número maior de variáveis, o que dificulta a interpretação.

In [ ]:
# SD 8: alvo numérico - colunas comportamentais - regras grandes (8) + punição 1.0

#  Standard Quality Function for numeric targets.

#     This quality function computes interestingness of subgroups based on
#     the difference between subgroup mean (or median) and dataset mean (or median),
#     weighted by the size of the subgroup raised to the power of 'a'.

#     Attributes:
#         a (float): Exponent to trade off between subgroup size and difference in means.
#         invert (bool): Whether to invert the quality function (not used currently).
#         estimator (str): Strategy for optimistic estimation ('sum', 'max', 'order').
#         centroid (str): Central tendency measure ('mean', 'median', 'sorted_median').

target = ps.NumericTarget("PURCHASES_TRX")
searchSpace = ps.create_selectors(
    creditData, ignore=["PURCHASES_TRX", "CUST_ID"]
)
print(searchSpace)
task = ps.SubgroupDiscoveryTask(
    creditData,
    target,
    searchSpace,
    result_set_size=10,
    depth=8,
    qf=ps.StandardQFNumeric(a=1.0, centroid='mean'),
)
result = ps.BeamSearch().execute(task)
result.to_dataframe()